In [1]:
import pandas as pd
from pyomo.environ import *

CN_file

In [2]:
# Read the excel file into a pandas DataFrame, change the file path to your own
file_name_FL2CH_CN = "C:\\Workspace\\Food_lost2Ch\\IO\\FL2CH_CN.xlsx"

# eta is yield
eta_df = pd.read_excel(file_name_FL2CH_CN, sheet_name = 'eta', index_col=0)

# FL is the qty of each Food loss or by-products stream
FL_df=pd.read_excel(file_name_FL2CH_CN, sheet_name = 'FL')

# CH is the demand qty of each platform chemical
CH_df=pd.read_excel(file_name_FL2CH_CN, sheet_name = 'CH')
# profit is the profit of each platform chemical
profit_df=pd.read_excel(file_name_FL2CH_CN, sheet_name = 'profit')
# GWP is the global warming potential saving of each platform chemical
GWP_df=pd.read_excel(file_name_FL2CH_CN, sheet_name = 'GWP')

US_file

In [3]:
# # Read the excel file into a pandas DataFrame, change the file path to your own
# file_name_FL2CH_US = "C:\\Workspace\\Food_lost2Ch\\IO\\FL2CH_US.xlsx"

# # eta is yield
# eta_df = pd.read_excel(file_name_FL2CH_US, sheet_name = 'eta', index_col=0)

# # FL is the qty of each Food loss or by-products stream
# FL_df=pd.read_excel(file_name_FL2CH_US, sheet_name = 'FL')

# # CH is the demand qty of each platform chemical
# CH_df=pd.read_excel(file_name_FL2CH_US, sheet_name = 'CH')
# # profit is the profit of each platform chemical
# profit_df=pd.read_excel(file_name_FL2CH_US, sheet_name = 'profit')
# # GWP is the global warming potential saving of each platform chemical
# GWP_df=pd.read_excel(file_name_FL2CH_US, sheet_name = 'GWP')

EU_file

In [4]:
# # Read the excel file into a pandas DataFrame, change the file path to your own
# file_name_FL2CH_EU = "C:\\Workspace\\Food_lost2Ch\\IO\\FL2CH_EU.xlsx"

# # eta is yield
# eta_df = pd.read_excel(file_name_FL2CH_EU, sheet_name = 'eta', index_col=0)

# # FL is the qty of each Food loss or by-products stream
# FL_df=pd.read_excel(file_name_FL2CH_EU, sheet_name = 'FL')

# # CH is the demand qty of each platform chemical
# CH_df=pd.read_excel(file_name_FL2CH_EU, sheet_name = 'CH')
# # profit is the profit of each platform chemical
# profit_df=pd.read_excel(file_name_FL2CH_EU, sheet_name = 'profit')
# # GWP is the global warming potential saving of each platform chemical
# GWP_df=pd.read_excel(file_name_FL2CH_EU, sheet_name = 'GWP')

In [5]:
# Convert the DataFrame to a dictionary
eta_dict = eta_df.to_dict(orient='index')
FL_dict = FL_df['Column1'].to_dict()
CH_dict = CH_df['Column1'].to_dict()
profit_dict = profit_df['Column1'].to_dict()
GWP_dict = GWP_df['Column1'].to_dict()

# Convert nested dictionary format for Pyomo
eta_dict = {(row, col): value for row, row_data in eta_dict.items() for col, value in row_data.items()}
FL_dict = {i+1: FL_dict[i] for i in FL_dict}
CH_dict = {i+1: CH_dict[i] for i in CH_dict}
profit_dict = {i+1: profit_dict[i] for i in profit_dict}
GWP_dict = {i+1: GWP_dict[i] for i in GWP_dict}

print(GWP_dict)

{1: 1422903.4852546917, 2: 0.0, 3: 8033410.0, 4: 0.0, 5: 7414999.999999999, 6: 6918300.0, 7: 1480560.0000000002, 8: 0.0, 9: 3792999.9999999995, 10: 1833999.9999999998, 11: 1789999.9999999998}


In [6]:
# Create a Concrete Model
model = ConcreteModel()

# Sets
model.I = RangeSet(1, 22) # for CN, (1, 22), for US (1, 13), for EU (1, 19)
model.J = RangeSet(1, 11)


# Parameters
model.FL = Param(model.I, initialize=FL_dict) 
model.CH = Param(model.J, initialize=CH_dict)     

model.eta = Param(model.I, model.J, initialize=eta_dict)

#option 1: model profit,choose this option when run the objective function 1
model.profit = Param(model.J, initialize=profit_dict)  
# # option 2: model GWP, choose this option when run the objective function 2
# model.GWP = Param(model.J, initialize=GWP_dict)

# Variables
model.x = Var(model.I, within=NonNegativeReals)
model.y = Var(model.J, within=NonNegativeReals)
model.w = Var(model.I, model.J, within=NonNegativeReals) # w[i,j] indicate the FL[i] used to produce CH[j]

# option 1: Objective function 1, profit
def objective_rule(model):
    return sum(model.y[j] * model.profit[j] for j in model.J)

# # option 2: Objective function 2, GWP
# def objective_rule(model):
#     return sum(model.y[j] * model.GWP[j] for j in model.J)

model.obj = Objective(rule=objective_rule, sense=maximize)

# Constraints
def x_constraint_rule(model, i):
    return model.x[i] == sum(model.w[i, j] for j in model.J)

model.x_constraint = Constraint(model.I, rule=x_constraint_rule)

def y_constraint_rule(model, j):
    return model.y[j] == sum(model.w[i, j] * model.eta[i, j] for i in model.I)

model.y_constraint = Constraint(model.J, rule=y_constraint_rule)

def y_demand_constraint_rule(model, j):
    return model.y[j] <= model.CH[j]

model.y_demand_constraint = Constraint(model.J, rule=y_demand_constraint_rule)

def x_fleet_constraint_rule(model, i):
    return model.x[i] <= model.FL[i]

model.x_fleet_constraint = Constraint(model.I, rule=x_fleet_constraint_rule)


# To solve the model, you would typically use a solver

# Solve the model
solvername='glpk'
solverpath_folder='C:\Python311\winglpk\glpk-4.65\w64' #does not need to be directly on c drive
solverpath_exe='C:\Python311\winglpk\glpk-4.65\w64\glpsol.exe' #does not need to be directly on c drive
# solver = SolverFactory('glpk')

solver=SolverFactory(solvername,executable=solverpath_exe)

results = solver.solve(model)

# Display the results
model.display()




Model unknown

  Variables:
    x : Size=22, Index=I
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          1 :     0 : 10144.2341500138 :  None : False : False : NonNegativeReals
          2 :     0 :              0.0 :  None : False : False : NonNegativeReals
          3 :     0 :              0.0 :  None : False : False : NonNegativeReals
          4 :     0 :              0.0 :  None : False : False : NonNegativeReals
          5 :     0 : 7749.54423606489 :  None : False : False : NonNegativeReals
          6 :     0 :              0.0 :  None : False : False : NonNegativeReals
          7 :     0 : 5.42237325058413 :  None : False : False : NonNegativeReals
          8 :     0 : 8127.25450901804 :  None : False : False : NonNegativeReals
          9 :     0 :  495.63502063146 :  None : False : False : NonNegativeReals
         10 :     0 : 707.654281879175 :  None : False : False : NonNegativeReals
         11 :     0 : 2201.09800924474 :  None : False 